In [1]:
f = open("../utils/ply-files/living room0.ply")
nickname = "hdroomfixed"

In [2]:
with f:
    lineNum = 0
    header = []
    body = []
    separators = []
    readBody = False
    for line in f:
        #print(f'Line[{lineNum}]: \t{line}')
        if(not readBody):
          header.append(line)
        else:
          body.append(line)
        lineNum += 1
        if line.startswith('end_header'):
          readBody = True
        if line.startswith('element'):
          parse = line.split(' ')
          separators.append({'name': parse[1], 'num': int(parse[2]) })

In [3]:
f.close()

In [4]:
sections = []
prev = 0
for i in range(len(separators)):
  sections.append({'name': separators[i]['name'], 'data': []})
  for l in body[prev:prev+separators[i]['num']]:
    l = l.strip('\n')
    l = l.strip('\t')
    fList = [float(x) for x in l.split(' ')]
    sections[i]['data'].append(fList)
  prev = separators[i]['num']

In [5]:
import numpy as np
import open3d as o3d
vertices = np.array(sections[0]['data'])
faces = np.array(sections[1]['data'])

IndexError: list index out of range

In [7]:
def displayVoxelCloud(cloud):
    pointCloudMatrix = np.asarray(cloud)
    pointCloudMatrix.shape
    pointCloudMatrix= pointCloudMatrix[:,0:3]
    pointCloudMatrix.shape
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
        
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
   # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    o3d.visualization.draw_geometries([vox_mesh]) #visualization

In [8]:
DECIMATION_CONSTANT = 5

decimated_vert = []

for i in range(len(vertices)):
  if(i % DECIMATION_CONSTANT == 0):
    decimated_vert.append(vertices[i])

print(len(decimated_vert))

displayVoxelCloud(decimated_vert)

75879


file_name = f"{nickname}-{DECIMATION_CONSTANT}mation"

In [14]:
np.savetxt(f"../utils/outputs/vertices-{file_name}.txt", decimated_vert)